In [1]:
import re
from bs4 import BeautifulSoup
import csv
from time import sleep
import requests
import pandas as pd

In [2]:
def scrape_suburb(url, need_sub):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the main unordered list with class 'pclist'
    pclist = soup.find('ul', class_='pclist')
    
    if not pclist:
        print("Could not find the pclist")
        return
    
    # Find all list items within the pclist
    postcode_items = pclist.find_all('li', recursive=False)
    
    # Prepare data for CSV
    data = []
    
    for index, item in enumerate(postcode_items):
        print(f"\nProcessing item {index}:")
    
        postcode_link = item.find('a')
        if postcode_link:
            postcode = postcode_link.text.strip()
            
            
            suburbs_ul = item.find('ul')
            if suburbs_ul:
                suburbs = suburbs_ul.find_all('li')
                
                
                for suburb in suburbs:
                    suburb_name = suburb.text.strip()
                    data.append([suburb_name, postcode])
                    
            else:
                print("No suburbs found for this postcode")
        else:
            print(f"Warning: No <a> tag found in item {index}")
    
    print(f"\nTotal entries extracted: {len(data)}")
    
    
    df = pd.DataFrame(data, columns=['Suburb', 'Postcode'])
    
    # If a suburb_list is provided, filter the DataFrame
    if need_sub:
        filtered_df = df[df['Suburb'].isin(need_sub)]
        print(f"\nFiltered DataFrame shape: {filtered_df.shape}")
    else:
        filtered_df = df
        print("\nNo filtering applied. Using full DataFrame.")
    
    # Write filtered data to CSV file
    if not filtered_df.empty:
        filtered_df['Postcode'] = filtered_df['Postcode'].astype(int)
        filtered_df = filtered_df[filtered_df['Postcode'] <= 4000]
        filtered_df.to_csv('../data/landing/victoria_suburbs_postcodes.csv', index=False)
        print("Filtered CSV file written successfully")
    else:
        print("No data to write, CSV file not created")
    
    # Print the first few entries for verification
    print("\nFirst 5 entries of the filtered data:")
    print(filtered_df.head())
    
    return filtered_df

In [3]:
need_sub = ['Albert Park', 'St Kilda', 'Armadale', 'Carlton North', 'Carlton', 'Parkville', 'Collingwood',
            'Abbotsford', 'Docklands', 'East Melbourne', 'St Kilda East', 'Elwood', 'Fitzroy', 
            'Fitzroy North', 'Clifton Hill', 'Flemington', 'Kensington', 'North Melbourne', 'West Melbourne', 
            'Port Melbourne','Prahran','Windsor','Richmond','Burnley','South Melbourne','South Yarra', 'Southbank', 
            'St Kilda', 'Toorak', 'Balwyn','Blackburn','Box Hill','Bulleen','Templestowe','Doncaster', 'Burwood',
            'Ashburton', 'Camberwell','Glen Iris','Canterbury','Surrey Hills','Mont Albert','Chadstone','Oakleigh',
            'Clayton','Doncaster East','Donvale East','Hawthorn','Glen Waverley','Mulgrave','Hawthorn','Kew','Mount Waverley',
            'Nunawading','Mitcham', 'Melbourne',
            'Vermont','Forest Hill','Burwood East','Aspendale','Chelsea','Carrum', 'Bentleigh', 'Brighton', 'Brighton East',
            'Carnegie','Caulfield','Cheltenham','Elsternwick','Hampton','Beaumaris','Malvern','Malvern East','Mentone','Parkdale','Mordialloc'
            ,'Murrumbeena','Hughesdale','Altona','Footscray','Keilor East','Avondale Heights','Melton','Newport','Spotswood', 
            'St Albans','Deer Park','Sunshine','Sydenham','Werribee','Footscray West','Williamstown','Yarraville','Seddon','Broadmeadows','Brunswick','Coburg' 
            ,'Brunswick East','Essendon','Tullamarine','Keilor Moonee Ponds', 'Ascot Vale', 
            'Oak Park','Glenroy','Fawkner','Pascoe Vale','Coburg North','Sunbury','Brunswick West','Bundoora','Greensborough','Hurstbridge',
            'Eltham','Research','Montmorency','Fairfield','Alphington','Heidelberg','Ivanhoe', 
           'Mill Park','Epping Northcote', 'Thomastown','Lalor','Thornbury','Whittlesea','Bayswater','Boronia', 
            'Croydon','Lilydale','Ferntree Gully','Ringwood','Rowville','Wantirna','Berwick','Cranbourne',
            'Dandenong','Dandenong North','Endeavour Hills','Narre Warren','Hampton Park','Noble Park','Pakenham','Springvale',
            'Dromana','Portsea','Frankston' ,'Hastings','Flinders','Mt Eliza','Mornington','Mt Martha','Seaford','Carrum Downs', 'Belmont','Grovedale' 
            'Corio' ,'Geelong','Newcombe' ,'Herne Hill','Geelong West', 'Lara', 'Newtown', 'North Geelong', 'Ballarat', 'Buninyong' ,
            'Sebastopol','Delacombe', 'Wendouree','Alfredton','Bendigo','Flora Hill','East Bendigo','Golden Square','Kangaroo Flat',
            'North Bendigo','Bairnsdale','Benalla','Castlemaine','Echuca','Hamilton','Horsham','Mildura','Moe','Newborough',
            'Morwell','Ocean Grove','Barwon Heads','Portland','Sale','Maffra','Seymour','Shepparton','Swan Hill','Torquay','Traralgon',
            'Wanagaratta','Warragul','Warrnambool','Wodonga']
            
            

In [5]:
url = 'https://postcodes-australia.com/state-postcodes/vic'
scrape_suburb(url, need_sub)

SSLError: HTTPSConnectionPool(host='postcodes-australia.com', port=443): Max retries exceeded with url: /state-postcodes/vic (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))